In [ ]:
!pip install --upgrade tensorflow-gpu

     |████████████████████████████████| 380.8MB 43kB/s 
     |████████████████████████████████| 3.8MB 38.4MB/s 
     |████████████████████████████████| 450kB 74.9MB/s 
     |████████████████████████████████| 81kB 12.8MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.2 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.10.0 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found exis

In [ ]:
import os
os.environ["KAGGLE_USERNAME"]="valeriianizolia"
os.environ['KAGGLE_KEY']='cac25c733910006ce9e1cef86202110f'
!kaggle competitions download -c chinese-char-recognition-smmo19

 95% 157M/165M [00:02<00:00, 40.9MB/s]
100% 165M/165M [00:03<00:00, 54.7MB/s]
 93% 172M/185M [00:02<00:00, 79.5MB/s]
100% 185M/185M [00:02<00:00, 89.6MB/s]
 90% 182M/202M [00:03<00:00, 46.9MB/s]
100% 202M/202M [00:03<00:00, 57.6MB/s]
 95% 185M/195M [00:04<00:00, 14.8MB/s]
100% 195M/195M [00:05<00:00, 40.7MB/s]
 95% 107M/113M [00:02<00:00, 27.8MB/s] 
100% 113M/113M [00:02<00:00, 44.3MB/s]
  0% 0.00/965k [00:00<?, ?B/s]
100% 965k/965k [00:00<00:00, 134MB/s]


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
!unzip \*.zip

Archive:  train-3.npy.zip
  inflating: train-3.npy             

Archive:  test.npy.zip
  inflating: test.npy                

Archive:  train-2.npy.zip
  inflating: train-2.npy             

Archive:  train-4.npy.zip
  inflating: train-4.npy             

Archive:  train-1.npy.zip
  inflating: train-1.npy             

5 archives were successfully processed.


In [ ]:
batch_size = 128
img_size = 32
val_size = 0.1
data_train = np.load("./train-1.npy", allow_pickle=True)
data_test = np.load("./test.npy", allow_pickle=True)
for i in range(2, 5):
  t = np.load(f"./train-{i}.npy", allow_pickle=True)
  data_train = np.concatenate([data_train, t])

tokens = np.unique([v[1] for v in data_train])
id_to_char = {id:char for id, char in enumerate(tokens)}
char_to_id = {char:id for id, char in enumerate(tokens)}


print(data_train.shape)
data_train = data_train.tolist()

(332987, 2)


In [ ]:
# нормализуем train данные
def preprocess_train(img, label):
  img = img / 255 - 0.5 
  img = tf.image.resize(
    img,
    (32,32),
    preserve_aspect_ratio=False,
    name=None
)
  return img, label

In [ ]:
def train_gen():
  for img, label in data_train[int(len(data_train)*val_size):]:
    img = img[..., None] # [batch, w, h, channels]
    yield img, char_to_id[label]

In [ ]:
ds_train = tf.data.Dataset.from_generator(train_gen, 
                                          output_types=(tf.float32, tf.int32), 
                                          output_shapes=((None,None,1), ())).map(preprocess_train, 
                                                                                 num_parallel_calls=-1).prefetch(-1).shuffle(1024).batch(batch_size).repeat()

In [ ]:
def val_gen():
  for img, label in data_train[:int(len(data_train)*val_size)]:
    img = img[..., None] # [batch, w, h, channels]
    yield img, char_to_id[label]

In [ ]:
ds_val = tf.data.Dataset.from_generator(val_gen, 
                                          output_types=(tf.float32, tf.int32), 
                                          output_shapes=((None, None,1), ())).map(preprocess_train, 
                                                                                 num_parallel_calls=-1).prefetch(-1).shuffle(1024).batch(batch_size).repeat()

In [ ]:
def test_gen():
  for img in data_test:
    img = img[..., None] # [batch, w, h, channels]
    yield img

In [ ]:
# нормализуем test данные
def preprocess_test(img):
  img = img / 255 - 0.5 
  img = tf.image.resize(
    img,
    (32,32),
    preserve_aspect_ratio=False,
    name=None
)
  return img

In [ ]:
ds_test = tf.data.Dataset.from_generator(test_gen, 
                                          output_types=(tf.float32), 
                                          output_shapes=(None, None,1)).map(preprocess_test, 
                                                                                 num_parallel_calls=-1).prefetch(-1).batch(batch_size)

In [ ]:
!tf_upgrade_v2 \
  --intree my_project/ \
  --outtree my_project_v2/ \
  --reportfile report.txt

TensorFlow 2.0 Upgrade Script
-----------------------------
Converted 0 files
Detected 0 issues that require attention
--------------------------------------------------------------------------------


Make sure to read the detailed log 'report.txt'



In [ ]:
NUM_EPOCHS=10
ens_models = 3

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=16, padding='same', kernel_size=(3,3), kernel_initializer="orthogonal",
                           input_shape=(32, 32, 1)),
    tf.keras.layers.LeakyReLU(0.1),
    tf.keras.layers.Conv2D(filters=32, padding="same", kernel_size=(3,3), kernel_initializer="orthogonal"),
    tf.keras.layers.LeakyReLU(0.1),
    tf.keras.layers.Conv2D(filters=64, padding='same', kernel_size=(3,3), kernel_initializer='orthogonal'),
    tf.keras.layers.LeakyReLU(0.1),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2), padding='same'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(filters=128, padding='same', kernel_size=(3,3),kernel_initializer='orthogonal'),
    tf.keras.layers.LeakyReLU(0.1),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2), padding='same'),
    tf.keras.layers.Conv2D(filters=256, padding='same', kernel_size=(3,3),kernel_initializer='orthogonal'),
    tf.keras.layers.LeakyReLU(0.1),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2), padding='same'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128),
    tf.keras.layers.LeakyReLU(0.1),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512),
    tf.keras.layers.LeakyReLU(0.1),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1000, activation='softmax')
])


# Model is the full model w/o custom layers
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(ds_train, epochs=NUM_EPOCHS, steps_per_epoch=10000, validation_data=ds_val, validation_steps=10000, shuffle=True,verbose=1)


print("Loss {}, Accuracy {}".format(loss, acc))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 10000 steps, validate on 10000 steps
Epoch 1/10
10000/10000 [==============================] - 478s 48ms/step - loss: 1.3417 - acc: 0.6721 - val_loss: 0.0767 - val_acc: 0.9818
Epoch 2/10
10000/10000 [==============================] - 472s 47ms/step - loss: 0.5121 - acc: 0.8506 - val_loss: 0.0477 - val_acc: 0.9900
Epoch 3/10
10000/10000 [==============================] - 483s 48ms/step - loss: 0.4254 - acc: 0.8754 - val_loss: 0.0387 - val_acc: 0.9918
Epoch 4/10
10000/10000 [==============================] - 494s 49ms/step - loss: 0.3885 - acc: 0.8865 - val_loss: 0.0286 - val_acc: 0.9937
Epoch 5/10
10000/10000 [==============================] - 484s 48ms/step - loss: 0.3677 - acc: 0.8932 - val_loss: 0.0242 - val_acc: 0.9950
Epoch 6/10
 9998/10000 [============================>.] - ETA: 0s - loss: 0.3497 - acc: 0.8982

KeyboardInterrupt: ignored

In [ ]:
pred = np.argmax(model.predict(ds_test, steps=10000), axis=1)
print(pred.shape)
pred

(83247,)


array([802, 846, 905, ..., 146, 263,  22])

In [ ]:
pred[1]

846

In [ ]:
l = []
for i in range(pred.shape[0]):
  pred[i] = id_to_char[pred[i]]
  l.append(pred[i])

In [ ]:
len(l)

83247

In [ ]:
ind = np.arange(1, data_test.shape[0]+1)
predictions = pd.DataFrame(ind, columns = ['Id'])
predictions["Category"]= l

In [ ]:
predictions

,Id,Category
0,1,63955
1,2,64432
2,3,64709
3,4,64177
4,5,61881
...,...,...
83242,83243,61877
83243,83244,64463
83244,83245,59837
83245,83246,60602


In [ ]:
predictions.to_csv("submission_chinese.csv", index=False)